In [96]:
import pandas as pd
import numpy as np
import torch
import os

from tqdm import tqdm

In [63]:
PARAMS = {
    0: 'drive',
    1: 'hpf',
    2: 'bias',
    3: 'output_db'
}

In [98]:
NUM_PARAMS = len(PARAMS)
AUDIO_LEN = 131_072
NUM_EXAMPLES = 1_000
PATH = "./data/"
CHECKPOINT_ID = "TEST"

In [99]:
FOLDER = os.path.join(PATH, CHECKPOINT_ID)

In [100]:
if not os.path.exists(FOLDER):
    os.makedirs(FOLDER)
    print(FOLDER, " was created!")

./data/TEST  was created!


In [65]:
def get_dummy_audio():
    return torch.rand(1, AUDIO_LEN)

In [66]:
def get_audio_features(x):
    return x[0,0].abs().item(), x[0,-1].abs().item()

In [67]:
def get_random_settings():
    return np.ones(NUM_PARAMS) * 0.5

In [68]:
def apply(signal, params):
    return signal * params[0]

In [69]:
def get_audio_embeddings(x, y):
    return torch.rand(1, 128), torch.rand(1, 128)

In [70]:
def predict_for_embeddings(z_x, z_y):
    # y_hat, p, z
    return torch.rand(1, AUDIO_LEN), torch.ones(NUM_PARAMS), torch.concat([z_x, z_y], dim=1)

In [71]:
def save_audio(signal, f_name):
    f_name = f"./audio{i}.wav"
    return f_name

In [72]:
def index_to_param_name(idx):
    return PARAMS.get(idx)

In [84]:
def param_vector_to_named_dictionary(vec, prefix=""):
    vec = vec.squeeze()
    
    if type(vec) == torch.Tensor:
        vec = vec.cpu().detach().numpy()
        
    assert(len(vec.shape) == 1)
    
    out = {}
    
    for i in range(len(vec)):
        param_name = index_to_param_name(i)
        out[f"{prefix}{param_name}"] = vec[i]
        
    return out

In [120]:
y_embeddings = []
metadata = []

x = get_dummy_audio()
x_b, x_d = get_audio_features(x)

for i in tqdm(range(NUM_EXAMPLES)):
    p = get_random_settings()
    y = apply(x, p)
    
    z_x, z_y = get_audio_embeddings(x, y)
    
    y_hat, p_hat, _ = predict_for_embeddings(z_x, z_y)
    
    y_hat_b, y_hat_d = get_audio_features(y_hat)
    
    f_name = save_audio(y_hat, i)   
    
    data_dict = {
        'id': i,
        'audio_file': f_name,
        'x_b': x_b,
        'x_d': x_d,
        'y_hat_b': y_hat_b,
        'y_hat_d': y_hat_d,
        'brightness_diff': y_hat_b - x_b,
        'depth_diff': y_hat_d - x_d
    }
    
    params = param_vector_to_named_dictionary(p, "p_")
    params_hat = param_vector_to_named_dictionary(p_hat, "p_hat_")
    
    data_dict.update(params)
    data_dict.update(params_hat)
    
    y_embeddings.append(z_y.cpu().detach().numpy())
    metadata.append(data_dict)

100%|██████████| 1000/1000 [00:01<00:00, 731.17it/s]


In [121]:
data_dict

{'id': 999,
 'audio_file': './audio999.wav',
 'x_b': 0.49158358573913574,
 'x_d': 0.3661285638809204,
 'y_hat_b': 0.671504020690918,
 'y_hat_d': 0.3070797324180603,
 'brightness_diff': 0.17992043495178223,
 'depth_diff': -0.05904883146286011,
 'p_drive': 0.5,
 'p_hpf': 0.5,
 'p_bias': 0.5,
 'p_output_db': 0.5,
 'p_hat_drive': 1.0,
 'p_hat_hpf': 1.0,
 'p_hat_bias': 1.0,
 'p_hat_output_db': 1.0}

In [122]:
embeddings = np.array(y_embeddings).squeeze()

In [123]:
embeddings.shape

(1000, 128)

In [124]:
df = pd.DataFrame(metadata).set_index('id')

In [125]:
np.save(os.path.join(FOLDER, "y_embeddings.npy"), embeddings)

In [126]:
df.to_csv(os.path.join(FOLDER, "metadata.csv"))